## Israel Elections 2020

data source: https://votes23.bechirot.gov.il/

In [7]:
import pandas as pd
import numpy as np

# read counted votes by city
df = pd.read_csv("https://media23.bechirot.gov.il/files/expc.csv", encoding='iso8859_8')
df.head(2)

,סמל ועדה,שם ישוב,סמל ישוב,בזב,מצביעים,פסולים,כשרים,אמת,ג,ודעם,...,נק,פה,ףז,ץ,ק,קי,קך,קץ,שס,Unnamed: 37
0,3,אור הגנוז,1294,281,184,0,184,2,83,0,...,0,1,0,0,0,0,0,0,58,NaN
1,17,אחווה,1157,185,143,0,143,13,0,0,...,0,88,0,0,0,0,0,0,2,NaN


In [8]:
# establish list of parties
parties = df.columns[7:-1]

# sum votes and sort
top10 = df[parties].sum().sort_values(ascending=False).head(10)
top10

מחל     1277097
פה      1150141
ודעם     564347
שס       336044
ג        264770
ל        254162
אמת      250402
טב       221237
נץ        18135
ז          3376
dtype: int64

In [9]:
# extract call letters of top parties
call10 = top10.index.values
call10

array(['מחל', 'פה', 'ודעם', 'שס', 'ג', 'ל', 'אמת', 'טב', 'נץ', 'ז'],
      dtype=object)

In [15]:
# top N cities with disqualified votes
df['disqual_ratio'] = df.eval('פסולים/מצביעים*100')
df[np.concatenate([['פסולים', 'כשרים', 'שם ישוב', 'מצביעים', 'disqual_ratio'], call10])].sort_values(by=['פסולים'], ascending=False).head(10)

,פסולים,כשרים,שם ישוב,מצביעים,disqual_ratio,מחל,פה,ודעם,שס,ג,ל,אמת,טב,נץ,ז
184,1621,260766,ירושלים,262387,0.617790,72516,32702,5304,44633,63313,6175,12824,20667,1784,176
363,997,266728,תל אביב יפו,267725,0.372397,58150,128591,11410,11093,2802,8584,38797,5909,556,318
242,929,84846,בני ברק,85775,1.083066,4951,1133,22,24389,51922,469,314,1200,366,15
178,863,144625,חיפה,145488,0.593176,38464,50081,14915,4218,5604,14893,10395,5020,356,154
1212,787,103423,מעטפות חיצוניות,104210,0.755206,34996,31188,5494,6857,3819,4142,7299,8402,505,216
162,785,98394,באר שבע,99179,0.791498,49291,17818,769,8808,1349,11726,3313,4411,453,87
160,694,117617,אשדוד,118311,0.586590,43072,18374,64,16994,13731,19414,2109,3123,397,58
208,646,130974,פתח תקווה,131620,0.490807,47074,37258,190,10787,6339,11412,5022,11849,558,118
480,636,109763,נתניה,110399,0.576092,47267,25906,109,13241,3602,10288,3265,5172,482,86
216,596,137894,ראשון לציון,138490,0.430356,55394,50944,190,6151,1166,12502,6268,4424,413,121


In [11]:
# top cities/places for נץ

df['n1'] = df.eval('נץ/מצביעים*100')
df[np.concatenate([['פסולים', 'כשרים', 'שם ישוב', 'מצביעים', 'n1'], call10])].sort_values(by=['n1'], ascending=False).head(10)

,פסולים,כשרים,שם ישוב,מצביעים,n1,מחל,פה,ודעם,שס,ג,ל,אמת,טב,נץ,ז
846,1,528,יצהר,529,23.251418,58,1,0,13,144,1,0,186,123,1
62,1,215,חברון,216,18.055556,45,2,0,7,21,1,0,98,39,0
105,1,122,נגוהות,123,12.195122,22,1,0,3,2,0,0,78,15,1
1094,1,485,כפר תפוח,486,10.905350,183,1,0,23,25,1,0,197,53,0
1202,2,444,שבי שומרון,446,10.089686,148,8,0,8,51,5,0,178,45,0
87,2,208,מבוא דותן,210,9.047619,103,15,0,8,10,4,4,43,19,0
807,2,714,אלון מורה,716,8.659218,108,1,0,20,27,1,1,494,62,0
399,1,619,בת עין,620,7.741935,242,5,0,41,71,0,1,204,48,5
126,0,66,עמיחי,66,7.575758,19,1,0,7,5,0,0,29,5,0
768,16,3268,קרית ארבע,3284,7.095006,1474,39,0,131,206,75,19,1075,233,1


In [28]:
# add column r1 for ratio of smillarity to overall results
# nature of r1 is still under question (how to compute)

he_valid = 'כשרים'
he_voters = 'מצביעים'
he_place = 'שם ישוב'

top10r = top10.divide(df[he_valid].sum())
def calcFit(r):
    return(r.filter(items=call10).divide(r[he_valid]) - top10r).abs().sum()

df['fit_index'] = df.apply(lambda x: calcFit(x), axis=1)
df[df[he_voters]>5000][np.concatenate([[he_place, he_voters, he_valid, 'fit_index'], call10])].sort_values(by=['fit_index'], ascending=True).head(20)

,שם ישוב,מצביעים,כשרים,fit_index,מחל,פה,ודעם,שס,ג,ל,אמת,טב,נץ,ז
1212,מעטפות חיצוניות,104210,103423,0.257452,34996,31188,5494,6857,3819,4142,7299,8402,505,216
178,חיפה,145488,144625,0.283188,38464,50081,14915,4218,5604,14893,10395,5020,356,154
217,רחובות,74296,73986,0.287390,25292,24095,257,5644,3283,5204,4854,4644,443,71
208,פתח תקווה,131620,130974,0.319802,47074,37258,190,10787,6339,11412,5022,11849,558,118
578,ירכא,6444,6325,0.331398,1860,1993,997,305,0,904,205,19,2,0
224,אזור,6730,6689,0.418793,2750,2148,24,739,76,317,339,231,33,2
270,חדרה,48251,47954,0.420267,21607,13046,214,4011,667,4648,1887,1498,180,36
350,ראש העין,33310,33216,0.426809,13336,11658,43,3146,740,1081,1325,1627,200,13
177,חולון,101584,101066,0.428842,42980,31303,163,10345,894,7081,4606,2914,364,92
695,חריש,5160,5135,0.430401,1669,1121,68,758,159,324,175,768,64,9
